In [1]:
#pip install -U sckit-learn

In [129]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [130]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [175]:
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [176]:
#Importing Dataset
data = 'Resources/final_mariokart_data.csv'
df = pd.read_csv (data)
df

,course_name,driver_id,name,special_skill,rarity,win_loss
0,DS Waluigi Pinball,1,Baby Mario,Normal,Boomerang Flower,1
1,SNES Mario Circuit 1T,1,Baby Mario,Normal,Boomerang Flower,1
2,3DS Cheep Cheep Lagoon,2,Baby Peach,Normal,Bubble,1
3,3DS Daisy Hills R/T,2,Baby Peach,Normal,Bubble,1
4,3DS Daisy Hills T,3,Baby Daisy,Normal,Bubble,1
...,...,...,...,...,...,...
535,3DS Neo Bowser City T,102,Waluigi (Bus Driver),High-End,Triple Bananas,0
536,Vancouver Velocity R,104,Wario (Hiker),High-End,Bob-omb Cannon,0
537,GBA Bowser's Castle 2T,105,Dry Bones (Gold),High-End,Coin Box,0
538,DS Airship Fortress R,108,Dry Bowser (Gold),High-End,Bob-omb Cannon,0


In [177]:
#Remove driver_id column
df.drop('driver_id', axis=1, inplace=True)
df

,course_name,name,special_skill,rarity,win_loss
0,DS Waluigi Pinball,Baby Mario,Normal,Boomerang Flower,1
1,SNES Mario Circuit 1T,Baby Mario,Normal,Boomerang Flower,1
2,3DS Cheep Cheep Lagoon,Baby Peach,Normal,Bubble,1
3,3DS Daisy Hills R/T,Baby Peach,Normal,Bubble,1
4,3DS Daisy Hills T,Baby Daisy,Normal,Bubble,1
...,...,...,...,...,...
535,3DS Neo Bowser City T,Waluigi (Bus Driver),High-End,Triple Bananas,0
536,Vancouver Velocity R,Wario (Hiker),High-End,Bob-omb Cannon,0
537,GBA Bowser's Castle 2T,Dry Bones (Gold),High-End,Coin Box,0
538,DS Airship Fortress R,Dry Bowser (Gold),High-End,Bob-omb Cannon,0


In [178]:
#Check the datatypes
df.dtypes

course_name      object
name             object
special_skill    object
rarity           object
win_loss          int64
dtype: object

In [179]:
#Set win_loss as target
inputs = df

In [180]:
#Add Encoder Library
from sklearn.preprocessing import LabelEncoder

In [181]:
#Identify columns to encode
le_course_name = LabelEncoder()
le_name = LabelEncoder()
le_special_skill = LabelEncoder()
le_rarity = LabelEncoder()

In [182]:
#Encode columns, associate a number to text
inputs['course_name_new'] = le_course_name.fit_transform(inputs['course_name'])
inputs['name_new'] = le_name.fit_transform(inputs['name'])
inputs['special_skill_new'] = le_name.fit_transform(inputs['special_skill'])
inputs['rarity_new'] = le_name.fit_transform(inputs['rarity'])
inputs.head()

,course_name,name,special_skill,rarity,win_loss,course_name_new,name_new,special_skill_new,rarity_new
0,DS Waluigi Pinball,Baby Mario,Normal,Boomerang Flower,1,46,2,1,3
1,SNES Mario Circuit 1T,Baby Mario,Normal,Boomerang Flower,1,147,2,1,3
2,3DS Cheep Cheep Lagoon,Baby Peach,Normal,Bubble,1,0,3,1,5
3,3DS Daisy Hills R/T,Baby Peach,Normal,Bubble,1,6,3,1,5
4,3DS Daisy Hills T,Baby Daisy,Normal,Bubble,1,7,0,1,5


In [183]:
#Remove course_name, name, special_skill & rarity
inputs_new = inputs.drop(['course_name', 'name', 'special_skill', 'rarity'],axis='columns')
inputs_new

,win_loss,course_name_new,name_new,special_skill_new,rarity_new
0,1,46,2,1,3
1,1,147,2,1,3
2,1,0,3,1,5
3,1,6,3,1,5
4,1,7,0,1,5
...,...,...,...,...,...
535,0,14,95,0,18
536,0,172,97,0,2
537,0,56,29,0,6
538,0,36,31,0,2


In [184]:
#Define the feature set
X = inputs_new.copy()
X = X.drop("win_loss", axis=1)
X.head()

,course_name_new,name_new,special_skill_new,rarity_new
0,46,2,1,3
1,147,2,1,3
2,0,3,1,5
3,6,3,1,5
4,7,0,1,5


In [185]:
#Define the target set
y = inputs_new["win_loss"].values
y[:4]

array([1, 1, 1, 1])

In [186]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [187]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(405, 4)
(135, 4)
(405,)
(135,)


In [188]:
# Splitting into Train and Test sets into an 80/20 split.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)

In [189]:
# Determine the shape of our training and testing sets.
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(432, 4)
(108, 4)
(432,)
(108,)


In [190]:
pip install termcolor

Note: you may need to restart the kernel to use updated packages.


In [191]:
import warnings 
warnings.filterwarnings("ignore")

In [192]:
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
from matplotlib import rcParams # figure size
from termcolor import colored as cl # text customization

from sklearn.tree import DecisionTreeClassifier as dtc # tree algorithm
from sklearn.model_selection import train_test_split # splitting the data
from sklearn.metrics import accuracy_score # model precision
from sklearn.tree import plot_tree # tree diagram

rcParams['figure.figsize'] = (25, 20)

In [193]:
inputs_new = dtc(criterion = 'entropy', max_depth = 3)
inputs_new.fit(X_train, y_train)

pred_model = model.predict(X_test)

print(cl('Accuracy of the model is {:.0%}'.format(accuracy_score(y_test, pred_model)), attrs = ['bold']))


Accuracy of the model is 76%


In [194]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [195]:
# Building  Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(criterion = 'entropy', random_state = 250)
rfc.fit(X_train, y_train)

# Evaluating on Training set
rfc_pred_train = rfc.predict(X_train)
print('Training Set Evaluation F1-Score=>',f1_score(y_train,rfc_pred_train))

Training Set Evaluation F1-Score=> 1.0


In [196]:
# Evaluating on Test set
rfc_pred_test = rfc.predict(X_test)
print('Testing Set Evaluation F1-Score=>',f1_score(y_test,rfc_pred_test))

Testing Set Evaluation F1-Score=> 0.6136363636363635


In [197]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [198]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [199]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [200]:
predictions

array([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0])

In [201]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,72,14
Actual 1,22,27


In [202]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [203]:
acc_score

0.7333333333333333

In [204]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,72,14
Actual 1,22,27


Accuracy Score : 0.7333333333333333
Classification Report
              precision    recall  f1-score   support

           0       0.77      0.84      0.80        86
           1       0.66      0.55      0.60        49

    accuracy                           0.73       135
   macro avg       0.71      0.69      0.70       135
weighted avg       0.73      0.73      0.73       135



In [205]:
#Add combination of numbers to predict win=1 or loss=0
model.predict([[172,97,0,2]])

array([0])

In [206]:
##ATTEMPTING TO CREATE DECISION TREE IMAGE

In [207]:
#pip install six

In [208]:
#from sklearn import tree

In [209]:
# Importing required packages for visualization
#from IPython.display import Image  
#from six import StringIO
#from sklearn.tree import export_graphviz
#import pydotplus, graphviz

In [212]:
# plotting tree with max_depth=3
#dot_data = StringIO()  

#export_graphviz(inputs_new, out_file=dot_data, filled=True, rounded=True,
                #feature_names=['course_name_new', "name_new", "special_skill_new", "rarity_new"], 
                #class_names=['win_loss', "1"])

#graph = pydotplus.graph_from_inputs_new_data(inputs_new_data.getvalue())
#Image(graph.tree_png())

In [213]:
#model = tree.DecisionTreeClassifier()

In [214]:
#model.fit(inputs_new,target)

In [215]:
#model.score(inputs_new,target)